In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
import os
from sklearn.ensemble import RandomForestClassifier


# Trova il percorso assoluto della cartella "esotic" (dove si trova il tuo ipynb)
current_dir = os.path.dirname(os.path.abspath("__file__"))

# Trova il percorso assoluto della cartella "lib"
lib_dir = os.path.join(current_dir, "..", "lib")

# Aggiungi il percorso di "lib" al PYTHONPATH
sys.path.append(lib_dir)

from Utility import *

# features = [
# 'RecentFormRatio', 'EloRatio'
# ]


features = [
'GoalRatioDifference', 'RecentFormDiff'
]

features2 = ['UltimoScontroDiretto', 'feat1', 'feat2']

classWeight={1: 3, 0: 1}

## DATA

In [3]:
data = getFullData().copy()
# data = pd.read_excel('../data/mergedSerieA.xlsx')

dataU = data.dropna(subset=features2)


c:\Users\vitto\OneDrive\Desktop\Personale\ML - Soccer\AwayToWin\..\lib\Utility.py:212: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['UltimoScontroDiretto'] = data['UltimoScontroDiretto'].replace(


## Feature Engineering

In [4]:
data = data.dropna(subset=features)
print(len(data), len(dataU))
# data = data.dropna()
print(len(data))
data['isOver'] = np.where(data['MatchGoal'] > 2.5, 1, 0)
data['isATW'] = np.where(((data['FTR'] == 'A')), 1, 0)
print('isATW:', round(100*data[data['isATW']==1]['isATW'].count() / len(data['isATW'])), '%')
x_train, x_test, y_train, y_test = train_test_split(
    data[features], data['isATW'], test_size=0.3, random_state=42, shuffle=True)#, stratify=data['isATS'])

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train = y_train.sort_index()
y_test = y_test.sort_index()


print(f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

38192 21165
38192
isATW: 29 %
X_train: (26734, 2) 
X_test: (11458, 2) 
y_train: (26734,) 
y_test: (11458,)


## Models

### Decision Tree model

In [5]:
from sklearn.tree import DecisionTreeClassifier



# Crea e addestra il modello di albero di decisione per la regressione
decTreeModel = DecisionTreeClassifier(random_state=42, min_samples_split=80
                                 , class_weight=classWeight
                                 # , class_weight='balanced'
                                 , max_depth=3  # , ccp_alpha= 0.00001
                                 # , max_features= 3
                                 ).fit(x_train, y_train)

# decTreeModel = RandomForestClassifier(random_state=42, n_estimators=100, 
#                                 min_samples_split=80, max_depth=3).fit(x_train_under, y_train_under) 

# dumpModel("Step1.joblib", decTreeModel)
# decTreeModel = loadModel("Step1.joblib")

### LogisticRegression

In [6]:
logRegModel = trainLogRegModel(x_train, y_train, class_weight=classWeight)

# dumpModel("Step1LogReg.joblib", logRegModel_u)
# rfModel = loadModel('Step1LogReg.joblib')

### Random Forest

In [7]:
# Crea e addestra il modello di Random Forest


from sklearn.model_selection import GridSearchCV

rfModel = RandomForestClassifier(random_state=42, n_estimators=200, 
                                min_samples_split=80, max_depth=3
                                # class_weight='balanced_subsample'
                                 , class_weight=classWeight
                                ).fit(x_train, y_train) 


# param_grid = {
#     'n_estimators': [200],
#     'min_samples_split': [50, 100],
#     'max_depth': [3, 5],
# }

# grid_search = GridSearchCV(estimator=rfModel, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train, y_train)

# print("Migliori iperparametri:", grid_search.best_params_)
# print("Migliore accuratezza:", grid_search.best_score_)

# best_rfModel = grid_search.best_estimator_

# rfModel = best_rfModel


# dumpModel("ATS-RF.joblib", rfModel)
# rfModel = loadModel('ATS-RF.joblib')

In [8]:
from sklearn.multiclass import OneVsRestClassifier

# Crea un classificatore One-vs-Rest usando Logistic Regression come classificatore base
ovr_classifier = OneVsRestClassifier(LogisticRegression())

# Addestra il classificatore
ovr_classifier.fit(x_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression())

## Utility Func for Gain

In [9]:
quotaMin = 1.4

def calculate_gain_1x(row, prediction):
    if (row[prediction] == 0):
        if row['Quota1x'] > quotaMin :
            if row['FTR'] != 'A':
                return row['Quota1x']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_ATW(row, prediction):
    if (row[prediction] == 1):
        if row['QuotaATW'] > quotaMin :
            if row['FTR'] == 'A':
                return row['QuotaATW']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_ATS(row, prediction):
    if (row[prediction] == 1):
        if row['ATS'] > quotaMin :
            if row['FTAG'] > 0:
                return row['ATS']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_BTS(row, prediction):
    if (row[prediction] == 1):
        if row['BTS'] > quotaMin :
            if row['FTHG'] > 0 and row['FTAG'] > 0:
                return row['BTS']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0
def calculate_gain_O25(row, prediction):
    
    if (row[prediction] == 1):
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    elif (row[prediction] == 0): 
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

def calculate_gain_Over25(row, prediction):
    # print('row', row['B365>2.5'], pd.isna(row['B365>2.5']))
    if (row[prediction] == 1):
        # return 0
        if row['B365>2.5'] > quotaMin :
            if row['MatchGoal'] > 2.5:
                return row['B365>2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0
    
def calculate_gain_Under25(row, prediction):
    if (row[prediction] == 0): 
        # return 0
        if row['B365<2.5'] > quotaMin :
            if row['MatchGoal'] < 2.5:
                return row['B365<2.5']-1
            else:
                return -1
        else:
            return 0
    else:
        return 0

## Metrics

In [10]:
from itertools import count
from math import nan
from sklearn.metrics import accuracy_score

df_train = data[data.index.isin(x_train.index)].copy()
print(len(df_train))

df_train['LogRegPred'] = logRegModel.predict(x_train)
df_train['DecTreePred'] = decTreeModel.predict(x_train)
df_train['RFPred'] = rfModel.predict(x_train)


df_test = data[data.index.isin(x_test.index)].copy()

print(len(df_test))

df_test['LogRegPred'] = logRegModel.predict(x_test)
df_test['DecTreePred'] = decTreeModel.predict(x_test)
df_test['RFPred'] = rfModel.predict(x_test)
df_test['1vsAllPred'] = ovr_classifier.predict(x_test)
df_test['FullPred'] = np.where(df_test['RFPred']+df_test['LogRegPred'] ==2, 1, 0)

print(
    f"Test LogReg Accuracy score: {round(100*accuracy_score(y_test, df_test['LogRegPred']),2)}%")
print(
    f"Test DecTree Accuracy score: {round(100*accuracy_score(y_test, df_test['DecTreePred']),2)}%")
print(
    f"Train Random Forest Accuracy score: {round(100*accuracy_score(y_train, df_train['RFPred']),2)}%")
print(
    f"Test Random Forest Accuracy score: {round(100*accuracy_score(y_test, df_test['RFPred']),2)}%")
print(
    f"Test OneVsAll Accuracy score: {round(100*accuracy_score(y_test, df_test['1vsAllPred']),2)}%")
print(
    f"Test FullPred Accuracy score: {round(100*accuracy_score(y_test, df_test['FullPred']),2)}%")


models = ['LogReg', 'RF', 'DecTree', '1vsAll']
rows = []
classAccuracyDf = pd.DataFrame(
    columns=['Modello', 'Accuratezza_Under', 'Accuratezza_Over'])
for model in models:
    row = {
        "Modello": model,
        "Accuratezza_Over": class_accuracy(y_test, df_test[model+'Pred'])['Over'],
        "Accuratezza_Under": class_accuracy(y_test, df_test[model+'Pred'])['Under']
    }
    rows.append(row)

classAccuracyDf = pd.DataFrame(rows)
classAccuracyDf.set_index('Modello', inplace=True)

print('')
print(classAccuracyDf)


# Ottieni le probabilità predette per ogni classe
probabilita = logRegModel.predict_proba(x_test)

# 
prediction = '1vsAllPred'
prediction = 'LogRegPred'
prediction = 'RFPred'
# prediction = 'DecTreePred'

df_test['QuotaATW'] = df_test['B365A'].fillna(df_test['AvgCA'].fillna(df_test['AvgA']))
df_test['QuotaH'] = df_test['B365H'].fillna(df_test['AvgCH'].fillna(df_test['AvgH']))
df_test['QuotaX'] = df_test['B365D'].fillna(df_test['AvgCD'].fillna(df_test['AvgD']))
df_test['Quota1x'] = 1/(1/df_test['QuotaH'] + 1/df_test['QuotaX'])

df_test['GainATW'] = df_test.apply(calculate_gain_ATW, axis=1, args=(prediction,) )
df_test['Gain1X'] = df_test.apply(calculate_gain_1x, axis=1, args=(prediction,) )

# df_test['Gain'] = df_test['GainOver'].loc(-1)+ df_test_under['GainUnder'].loc(-1) 
# df_test['Equity'] = df_test['Gain'].cumsum()
df_test['EquityATW'] = df_test['GainATW'].cumsum()
df_test['Equity1x'] = df_test['Gain1X'].cumsum()
df_test['BetNum'] =  df_test['GainATW'][df_test['GainATW'] != 0].count()
df_test['RowNum'] =  df_test['GainATW'].count()

# print(df_test['Equity'].tail(1))
print(df_test['EquityATW'].tail(1))
print('Equity1x', df_test['Equity1x'].tail(1))
print('Bet number:', df_test['BetNum'].tail(1), 'on', df_test['RowNum'].tail(1))

from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(df_test['BTS'], df_test[prediction])
# print(cm)


# exportExcelWithTimeStamp(df_test, "../Dataframe/", "df.xlsx")

26734
11458
Test LogReg Accuracy score: 52.44%
Test DecTree Accuracy score: 52.6%
Train Random Forest Accuracy score: 59.23%
Test Random Forest Accuracy score: 58.66%
Test OneVsAll Accuracy score: 70.97%
Test FullPred Accuracy score: 58.72%

         Accuratezza_Over  Accuratezza_Under
Modello                                     
LogReg           0.361789           0.818784
RF               0.385989           0.793930
DecTree          0.362036           0.816949
1vsAll           0.582834           0.715524
51638   -372.09
Name: EquityATW, dtype: float64
Equity1x 51638   -11.035135
Name: Equity1x, dtype: float64
Bet number: 51638    5626
Name: BetNum, dtype: int64 on 51638    11458
Name: RowNum, dtype: int64


In [11]:
# Unisci i due DataFrame verticalmente
complete_df = pd.concat([df_train, df_test])
exportExcelWithTimeStamp(complete_df, "../Dataframe/", "df.xlsx")